# Supply Points (Case dell'Acqua) Data Preprocessing

In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Paths

In [2]:
dir_store_path = "/Users/massimilianoarca/Documents/PoliMi/Research Grant/SafeCREW/Data/Milano/temporary results"
root_folder_path = "/Users/massimilianoarca/Library/CloudStorage/OneDrive-PolitecnicodiMilano/SafeCREW/soft_sensors/Soft Sensor CS2Milan"

grab_samples_supply_points_path = os.path.join(
    root_folder_path,
    "Case dell'acqua - Grab Samples (main)/0. Case acqua - 2010-2023.xlsx",
)
all_grab_samples_path = os.path.join(
    root_folder_path, "Tutti punti - Grab Samples"
)

## Collect all grab samples for Supply Points

### Get Name and ID of Supply Points

In [3]:
column_list = "CS, CT"

meta_supply_points_df = pd.read_excel(
    grab_samples_supply_points_path, usecols=column_list, header=4
)

In [4]:
meta_supply_points_df

,filtro 1,filtro 2
0,C.A. ingresso Menotti > U.di Nemi,ING_UCCNEMI
1,Casa Acqua Appennini con CO2,SII00801
2,Casa Acqua Lessona,SII00659
3,Casa Acqua Lessona,SII00659
4,Casa Acqua Via Appennini non trattata,SII00802
...,...,...
2263,Casa dell'acqua Piazza Scolari,HOUSE_SCOLARI
2264,V.le Omero-NT-CA21,HOUSE_OME1
2265,Menotti > U.di Nemi-NT-CA08,HOUSE_UCC1
2266,Casa dell'acqua Piazza Ovidio,HOUSE_OVIDIO


### Load all grab samples

In [5]:
column_list = [
    "Data di prelievo",
    "Rapporto di prova",
    "Punto di prelievo",
    "Codice punto di prelievo",
    "Campagna",
    "Analisi programmate",
    "ZONA",
    "Alcalinità (mg/L)",
    "Alcalinità equivalente a carbonati (mg/L di CO3)",
    "Alcalinità equivalente a idrossidi (mg/L di OH)",
    "Bicarbonati (mg/L)",
    "Torbidità (NTu)",
    "Colore (Cu)",
    "Conduttività a 20°C (µS/cm)",
    "Concentrazione ioni idrogeno (unità pH)",
    "Durezza (da calcolo) (°F)",
    "Durezza totale (°F)",
    "Indice di aggressività ottenuto per calcolo (no unità)",
    "Residuo secco a 180°C (mg/L)",
    "Solidi sospesi totali (mg/L)",
    "Cloro residuo libero (mg/L di Cl2)",
    "Cloro residuo libero (al prelievo) (mg/L di Cl2)",
    "Concentr. ioni idrogeno (al prelievo) (unità pH)",
    "Temperatura - °C",
    "Temperatura (al prelievo) (°C)",
    "Temperatura (°C)",
    "TOC - carbonio organico totale (mg/L di C)",
    "Cloroformio (µg/L)",
    "Bromodiclorometano (µg/L)",
    "Dibromoclorometano (µg/L)",
    "Bromoformio (µg/L)",
    "Sommatoria totale trialometani (µg/L)",
    "Conta delle colonie a 22°C (UFC/mL)",
    "Conteggio colonie a 30°C (UFC/mL)",
    "Conta delle colonie a 37°C (UFC/mL)",
    "Batteri coliformi a 37°C (MPN/100 mL)",
    "Enterococchi (MPN/100 mL)",
    "Escherichia coli (MPN/100 mL)",
    "Pseudomonas aeruginosa (UFC/250 mL)",
    "Pseudomonas aeruginosa (UFC/100 mL)",
    "Legionella spp (UFC/L)",
    "Legionella pneumophila (UFC/L)",
    "Legionella molecolare pneumophila",
    "Legionella molecolare spp",
]

In [6]:
grab_samples = []

for file in os.listdir(all_grab_samples_path):
    if file.endswith(".xlsx"):
        df = pd.read_excel(os.path.join(all_grab_samples_path, file), header=11)
    else:
        df = pd.read_excel(os.path.join(all_grab_samples_path, file), header=15)
    common_cols = list(set(df.columns) & set(column_list))
    df = df[common_cols]
    grab_samples.append(df)

grab_samples_df = pd.concat(grab_samples)

### Get Supply Points Grab Samples

In [7]:
supply_points_df = grab_samples_df.merge(
    meta_supply_points_df,
    left_on="Codice punto di prelievo",
    right_on="filtro 2",
    how="left",
)

In [8]:
# supply_points_df.drop(columns=["filtro 1", "filtro 2"], inplace=True)
supply_points_df.drop_duplicates(inplace=True)

In [9]:
# combine the different Temperature columns into one column by mantaining the positions without deleting the nans and drop the others
supply_points_df["Temperatura °C"] = supply_points_df["Temperatura (°C)"]

for col in ["Temperatura - °C", "Temperatura (al prelievo) (°C)"]:
    supply_points_df["Temperatura °C"] = supply_points_df[
        "Temperatura °C"
    ].combine_first(supply_points_df[col])

supply_points_df.drop(
    columns=[
        "Temperatura - °C",
        "Temperatura (al prelievo) (°C)",
        "Temperatura (°C)",
    ],
    inplace=True,
)

In [10]:
supply_points_df = supply_points_df[
    (
        supply_points_df["filtro 1"].notna()
        | supply_points_df["filtro 2"].notna()
    )
]

In [14]:
supply_points_df.to_excel(
    os.path.join(dir_store_path, "All supply points.xlsx"),
    index=False,
)

In [15]:
supply_points_df

,Cloroformio (µg/L),Conteggio colonie a 30°C (UFC/mL),Campagna,TOC - carbonio organico totale (mg/L di C),Pseudomonas aeruginosa (UFC/100 mL),Bromodiclorometano (µg/L),Concentrazione ioni idrogeno (unità pH),Residuo secco a 180°C (mg/L),ZONA,Alcalinità (mg/L),...,Durezza (da calcolo) (°F),Legionella pneumophila (UFC/L),Legionella molecolare spp,Legionella spp (UFC/L),Concentr. ioni idrogeno (al prelievo) (unità pH),Legionella molecolare pneumophila,Cloro residuo libero (al prelievo) (mg/L di Cl2),filtro 1,filtro 2,Temperatura °C
6117,NaN,NaN,CASA_ACQUA,NaN,NaN,NaN,7.8,313.874,Case acqua,153.07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C.A. ingresso Menotti > U.di Nemi,ING_UCCNEMI,18.7
7288,NaN,NaN,UTENZA,NaN,NaN,NaN,5.8,326.0,case acqua,95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Casa Acqua Appennini con CO2,SII00801,10.3
7290,NaN,NaN,MICRO_PS,NaN,NaN,NaN,NaN,NaN,Case acqua,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Casa Acqua Lessona,SII00659,NaN
7301,NaN,NaN,MICR_UT_PS,NaN,NaN,NaN,NaN,NaN,case acqua,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Casa Acqua Lessona,SII00659,NaN
7312,NaN,NaN,UTENZA,NaN,NaN,NaN,7.5,320.0,case acqua,122,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Casa Acqua Via Appennini non trattata,SII00802,11.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564848,NaN,NaN,DKR,NaN,NaN,NaN,NaN,NaN,-,NaN,...,32.6,NaN,NaN,NaN,7.3,NaN,NaN,Casa dell'acqua Piazza Ovidio,HOUSE_OVIDIO,NaN
564860,NaN,NaN,DKR,NaN,NaN,NaN,NaN,NaN,-,NaN,...,33.6,NaN,NaN,NaN,8.0,NaN,NaN,Casa dell'acqua Piazzale Cuoco,HOUSE_CUOCO,NaN
564891,NaN,NaN,DKR,NaN,NaN,NaN,NaN,NaN,-,NaN,...,26.9,NaN,NaN,NaN,7.9,NaN,NaN,Casa dell'acqua Monte Peralba,HOUSE_MONTE_PERALBA,NaN
564905,NaN,NaN,DKR,NaN,NaN,NaN,NaN,NaN,-,NaN,...,28.3,NaN,NaN,NaN,7.7,NaN,NaN,Casa dell'acqua Via Tabacchi,HOUSE_TABACCHI,NaN


In [16]:
columns_list = "A:G, M:Y, BY, BZ:CB, CD, CF, CJ, CL:CM"

In [17]:
raw_grab_samples_df = pd.read_excel(
    grab_samples_supply_points_path, header=4, usecols=columns_list
)

In [18]:
def convert_to_float(x):
    try:
        if x == "3,0":
            pass
        return float(x.replace(",", ".")) if type(x) == str and "," in x else x
    except ValueError:
        return x

In [19]:
raw_grab_samples_df["Carica batterica a 22°C (UFC/mL)"] = raw_grab_samples_df[
    "Carica batterica a 22°C (UFC/mL)"
].apply(convert_to_float)

In [20]:
raw_grab_samples_df["Codice punto di prelievo"] = raw_grab_samples_df[
    "Codice punto di prelievo"
].str.strip()

In [21]:
raw_grab_samples_df

,Data di prelievo,Rapporto di prova,Punto di prelievo,Codice punto di prelievo,Campagna,Analisi programmate,ZONA,Alcalinità (mg/L),Cloro residuo libero (mg/L di Cl2),Colore (CU),...,Bicarbonati - mg/L,TOC - carbonio organico totale (mg/L di C),Carica batterica a 22°C (UFC/mL),Conteggio colonie a 30°C (UFC/mL),Carica batterica a 37°C (UFC/mL),Batteri coliformi a 37°C (MPN / 100 mL),Enterococchi (MPN / 100mL),Escherichia Coli (MPN / 100mL),Pseudomonas aeruginosa (UFC / 250mL),Pseudomonas aeruginosa (UFC/100 mL)
0,2014-07-18,3185/14,C.A. ingresso Menotti > U.di Nemi,ING_UCCNEMI,CASA_ACQUA,Analisi case dell'acqua,Case acqua,153.07,0.06,<0.01,...,NaN,NaN,342,NaN,845,1,0,0,1,NaN
1,2016-06-13,2818/16,Casa Acqua Appennini con CO2,SII00801,UTENZA,"CBT 22° e 37°, Coliformi, E-Coli, Enterococchi...",case acqua,95,0.01,0.77,...,94.0,NaN,<1,NaN,<1,<1,<1,<1,NaN,NaN
2,2014-01-03,16/14,Casa Acqua Lessona,SII00659,MICRO_PS,"Coliformi, E-Coli, Enterococchi, Pseudomonas",Case acqua,NaN,0.05,NaN,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN
3,2016-04-22,1860/16,Casa Acqua Lessona,SII00659,MICR_UT_PS,"CBT 22° e 37°, Coliformi, E-Coli, Enterococchi...",case acqua,NaN,0.03,NaN,...,NaN,NaN,2,NaN,<1,<1,<1,<1,<1,NaN
4,2016-06-13,2819/16,Casa Acqua Via Appennini non trattata,SII00802,UTENZA,"CBT 22° e 37°, Coliformi, E-Coli, Enterococchi...",case acqua,122,0.02,0.77,...,122.0,NaN,<1,NaN,<1,<1,<1,<1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2263,2023-03-29,1554/23,Casa dell'acqua Piazza Scolari,HOUSE_SCOLARI,DKR,Casa Dkr,Case Acqua,NaN,NaN,NaN,...,NaN,NaN,< 1,NaN,NaN,0,0,0,< 1,NaN
2264,2023-03-30,1573/23,V.le Omero-NT-CA21,HOUSE_OME1,PROACQUA,Case Proacqua,Case Acqua,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2265,2023-03-30,1574/23,Menotti > U.di Nemi-NT-CA08,HOUSE_UCC1,PROACQUA,Case Proacqua,Case Acqua,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2266,2023-03-30,1575/23,Casa dell'acqua Piazza Ovidio,HOUSE_OVIDIO,DKR,Casa Dkr,Case Acqua,NaN,NaN,NaN,...,NaN,NaN,< 1,NaN,NaN,0,0,0,<1,NaN


In [25]:
raw_grab_samples_df.to_excel(
    os.path.join(dir_store_path, "raw_grab_samples_supply_points.xlsx"),
    index=False,
)